### Script 2 - Smoothing algorithm (before event cleaning)

<b>Function</b>: smoothing isolated classifications. 

<b>Criterion</b>: 3 before/after (1112111) (requires event before and after)

<u>Output files</u>:
<br>
Smoothed file (600 TRs; step2_smooth.txt file)
<br>
Index file that shows which TR’s smoothed (00000010000010000) (in notebook)
<br>
% TR’s smoothed (single digits?) = sum(indexfile)/length (in notebook) – subj loop, average across subjects

In [1]:
import os

In [ ]:
# set root directory
root_dir = "/Path/To/EMBODY_Study/Files"

In [3]:
# set analysis name
analysis = "phase1_demo"

In [4]:
# define subjects
subjects = [124]

In [5]:
def imports_subj_meditation(subj):

    # change directory to subject file
    os.chdir("%s/%s/results/step2/%d/"
             % (root_dir, analysis, subj))

    # read in subject's text file
    filename = "step2_3cat.txt"

    return filename

In [6]:
def read_txt_file(filename):
    """Read meditation file, returns array of digits
        filename: input is string (e.g., "step2_3cat.txt")
    """

    with open(filename, 'rw') as f:
        content = f.readlines()
        content = list(content[0])

        # filter items in content by whether or not they are a digit. if digit, convert to integer.
        content = [int(x) for x in content if x.isdigit()]

        return content

In [7]:
def write_txt_file(filename, to_write):

    with open(filename, 'w') as f:

        f.write(",".join(to_write))

In [8]:
# Count the number of events per condition & smooth noise within events
def event_smoothing(meditation): 
    """
    Smooth over noise within events by changing noise (an event <= 3TR surrounded within an event that is at least 
    3TR long before and after the 'noise' event.
    
    Outputs a new array of smoothed events.
    """
    
    # smoothing index list
    smoothing_indices = [0] * 600
    
    # create new array for smoothed events
    smooth_event = []
            
    # Iterate through meditation array
    for index in range(len(meditation) - 3):
        
        current_value = meditation[index]
        
        next_1 = meditation[index + 1]
        next_2 = meditation[index + 2]
        next_3 = meditation[index + 3]
        
        prev_1 = meditation[index - 1]
        prev_2 = meditation[index - 2]
        prev_3 = meditation[index - 3]
        
        # check for values to be smoothed in first three indicies
        if index == 2:
            if current_value != next_1:
                if (next_1 == next_2 and
                    next_2 == next_3):
                                        
                    smooth_event.append(next_1)
                    # note that index was smoothed in smoothing_indices vector
                    smoothing_indices[index] = 1
    
            else:
                smooth_event.append(current_value)
        
        # check for values to be smoothed in first three indicies 
        elif index == 596:

            if ((prev_1 == prev_2) and (prev_2 == prev_3)) and \
                ((next_1 == next_2) and (next_2 == next_3)) and \
                (current_value != next_1) and (next_1 == prev_1):
                
                for i in range(4):
                    smooth_event.append(next_1)
            
            else:
                for i in range(596, 600):
                    smooth_event.append(meditation[i])
        
        # check for values to be smoothed in the rest of the values
        elif  ( index > 3 and
             (current_value != prev_1) and
             (current_value != next_1) and
             (next_1 == next_2) and
             (next_2 == next_3) and
             (prev_1 == prev_2) and
             (prev_2 == prev_3) and
             (prev_1 == next_1)
             ):

            smooth_event.append(next_1)
            smoothing_indices[index] = 1

        else:
            smooth_event.append(current_value)

                        
    return (smooth_event, smoothing_indices)
            

In [9]:
# Smooth participant data and calculate percent of data that has been smoothed.
# Display percent of participant data smoothed for each participant and the mean percent smoothed over all subjects.


percent_smoothed_list = []

for subj in subjects:
    
    write_path = "%s/%s/results/step2/%d/" % (root_dir, analysis, subj)
    print(write_path)
    os.chdir(write_path)
        
    # read in meditation if meditation is txt_file
    meditation = read_txt_file(imports_subj_meditation(subj))
    
    smooth_events = event_smoothing(meditation)[0]
    smoothing_indices = event_smoothing(meditation)[1]
    
    percent_smoothed = sum(smoothing_indices)/float(len(smoothing_indices)) * 100
    percent_smoothed_list.append(float(percent_smoothed))
    percent_smoothed = '{:.1f}%'.format(percent_smoothed)

    
    smooth_events = [str(x) for x in smooth_events]
    smoothing_indices = [str(x) for x in smoothing_indices]
    
    print("Subject: %d" % subj)
    print("")
    print("Percent Smoothed: %s" % percent_smoothed)
    print("")
    print("Writing 'step2_smooth.txt")
    write_txt_file("step2_smooth.txt", smooth_events)
    print("Writing 'step2_smooth_indices.txt")
    write_txt_file("step2_smooth_indices.txt", smoothing_indices)
    print("")
    print("-----")
    print("")

mean_percent_smoothed = sum(percent_smoothed_list)/float(len(percent_smoothed_list))
print("")
print("Mean Percent Smoothed: {:.1f}%".format(mean_percent_smoothed))


/Users/sashaskinner/Documents/Projects/embody_study/publication_scripts/embody_demo/embody_study//phase1_demo/results/step2/124/
Subject: 124

Percent Smoothed: 1.5%

Writing 'step2_smooth.txt
Writing 'step2_smooth_indices.txt

-----


Mean Percent Smoothed: 1.5%
